In [ ]:
import underworld as uw
import math
from underworld import function as fn
import glucifer.pylab as plt

#linearMesh = uw.mesh.FeMesh_Cartesian(('Q2','dPc1'), (32,32), (0.,0.), (1.,1.))
linearMesh = uw.mesh.FeMesh_Cartesian('Q2,Q1', (32,32), (0.,0.), (1.,1.))
constantMesh = linearMesh.subMesh

velocityField = uw.fevariable.FeVariable(linearMesh,2)
velocityField.data[:] = (0.,0.)

pressureField = uw.fevariable.FeVariable(constantMesh,1)
pressureField.data[:] = 0.

# freeslip
IWalls = linearMesh.specialSets["MinI_VertexSet"] + linearMesh.specialSets["MaxI_VertexSet"]
JWalls = linearMesh.specialSets["MinJ_VertexSet"] + linearMesh.specialSets["MaxJ_VertexSet"]
freeslip = uw.conditions.DirichletCondition(velocityField, (IWalls, JWalls))


solA = fn.analytic.SolCx()
# solA = fn.analytic.SolKx()
#solA = fn.analytic.SolKz()

In [ ]:
stokesSystem = uw.systems.Stokes(velocityField,pressureField,solA.viscosityFn,solA.bodyForceFn,conditions=[freeslip,], rtolerance=1.e-5)


In [ ]:
# can optionally set penalty this way
solver=uw.systems.Solver(stokesSystem, penalty=1.0)

In [ ]:

solver.options.A11.ksp_rtol=1e-4
solver.options.scr.ksp_rtol=1e-3

solver.options.main.Q22_pc_type='uwscale'

In [ ]:
solver.options.mg.active=False
solver.options.A11.set_direct()
solver.options.A11.list()

In [ ]:
from libUnderworld import petsc
petsc.OptionsPrint()

In [ ]:
solver.solve()

In [ ]:
# can set penalty dynamically this way
solver.options.main.penalty=0.5
solver.solve()

In [ ]:
solver.options.mg_accel.list()

In [ ]:
solver.options.main.list()
solver.options.mg_accel.mg_accelerating_smoothing=True

In [ ]:
# don't have to specify penalty again. Will use currently set value.
solver.solve()

In [ ]:
solver.options.mg_accel.mg_smoothing_adjust_on_convergence_rate=True

In [ ]:
solver.solve()

In [ ]:
from libUnderworld import petsc

In [ ]:
#petsc.OptionsClear()

In [ ]:
petsc.OptionsPrint()

In [ ]:
solver.options.main.Q22_pc_type='uw'
solver.solve(penalty=0)

In [ ]:
solver.options.main.change_backsolve=True
solver.options.backsolveA11.ksp_view=None

In [ ]:
solver.solve()

In [ ]:
solver.options.backsolveA11.list()

In [ ]:
solver.options.backsolveA11.help()
